# Lab 2: Parallel spatial analysis
### Objective:
Learn how to calculate parallel spatial autocorrelation (Moran’s I)
### Successful outcome:
You will calculate a spatial autocorrelation index, based on serial and parallel approaches.

This is link to [Google](http://www.google.com)

## Step 1: Install packages 

In [1]:
ipak <- function(pkg){
new.pkg <- pkg[!(pkg %in% installed.packages()[, "Package"])]
if (length(new.pkg)) 
    install.packages(new.pkg, dependencies = TRUE)
sapply(pkg, require, character.only = TRUE)
}

# usage
packages <- c("snow", "foreach", "doSNOW",  "parallel", "boot", "maptools", "spdep")
ipak(packages)

Loading required package: snow
Loading required package: foreach
Loading required package: doSNOW
Loading required package: iterators
Loading required package: parallel

Attaching package: ‘parallel’

The following objects are masked from ‘package:snow’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, clusterSplit, makeCluster, parApply,
    parCapply, parLapply, parRapply, parSapply, splitIndices,
    stopCluster

Loading required package: boot
Loading required package: maptools
Loading required package: sp
Checking rgeos availability: TRUE
Loading required package: spdep
Loading required package: Matrix


snow  foreach   doSNOW parallel     boot maptools    spdep 
    TRUE     TRUE     TRUE     TRUE     TRUE     TRUE     TRUE

## Step 2: Moran’s I: An index of autocorrelation

In [ ]:
#Change to your username
username <- "mhjeong"

In [ ]:
mydir <- paste("/mnt/jhub/users/", username, sep ="")

In [ ]:
mydir
#an absolute file path representing the current working directory of the R process
setwd(mydir)

In [ ]:
# Confirm the working directory is set to what you wanted:
getwd()

In [ ]:
#Set up a shape folder
setwd("/mnt/jhub/shape/")

In [ ]:
#Get shapefile header information in the radiation data
getinfo.shape("Oct_17_20_proj.shp")
#Reads data from a points shapefile
radiation<-readShapePoints ("Oct_17_20_proj.shp")

#Retrieve spatial coordinates from a Spatial object
coords<-coordinates(radiation)
IDs<-row.names(as(radiation, "data.frame"))

#Neighbourhood contiguity by distance
radiation_nei<-dnearneigh(coords, d1=0, d2=20, row.names=IDs)

#Spatial weights for neighbours lists
radiation_nbq_wb<-nb2listw(radiation_nei, style="W")


In [ ]:
#Moran's I test for spatial autocorrelation for a given dataset and W matrix
#field_5 indicates gamma-ray counts as a given dataset.
#radiation_nbq_wb is W matrix.
#This test approximates the test statistics by a normal distribution.
moran.test(radiation$field_5, listw=radiation_nbq_wb)


## Step 3: Moran’s I: Permutation-based hypothesis testing based on parallel  implementation

In [ ]:
#gamma-ray counts 
gamma <- radiation$field_5
#spatial weight matrix
listw <- radiation_nbq_wb
#simulation numbers
nsim <- 999
n <- length(listw$neighbours)
S0 <- Szero(listw)

In [ ]:
#cluster using 10 processes
cluster = makeCluster(10, type = "SOCK")
registerDoSNOW(cluster)
#share data sets
clusterExport(cluster, c("gamma", "listw","n","S0"))
ptm <- proc.time()
results = foreach(n = 1:nsim, .combine = c) %dopar% {
 library(spdep); moran(sample(gamma), listw, n, S0,zero.policy=NULL)$I
}
proc.time() - ptm

stopCluster(cluster)

In [ ]:
#Calculate Moran's I
paMoran <- function(res, x, listw, nsim,zero.policy=NULL,alternative="greater") {
  n <- length(listw$neighbours)
  S0 <- Szero(listw)

  res[nsim+1] <- moran(x, listw, n, S0, zero.policy)$I
  rankres <- rank(res)
  xrank <- rankres[length(res)]
	diff <- nsim - xrank
	diff <- ifelse(diff > 0, diff, 0)

  if (alternative == "less") 
        	pval <- punif((diff + 1)/(nsim + 1), lower.tail=FALSE)
    	else if (alternative == "greater") 
        	pval <- punif((diff + 1)/(nsim + 1))
  
	statistic <- res[nsim+1]
	names(statistic) <- "statistic"
	parameter <- xrank
	names(parameter) <- "observed rank"
	method <- "Parallel Monte-Carlo simulation of Moran's I"
    names(pval) <- "p-value"        
	lres <- list(statistic=statistic, parameter=parameter,
	    p.value=pval, alternative=alternative, method=method,res=res)  
  	lres 
}
      
#example 1
mtest <- paMoran(results,gamma,listw,nsim)
mtest$method
mtest$statistic 
mtest$parameter
mtest$p.value
#mtest$res

## Q1: Are there any differences for Moran's I when the number of random permutations is increased such as 1999?

In [ ]:
# Answer, explain your solution